# Numeric Feature Normalization, Scaling, and binning in Spark

In [1]:
%%init_spark
launcher.master="yarn"

## StandardScaler
The StandardScaler is an estimator and takes a column of feature vector and transforms it to have a unit standard deviation and/or zero mean. You can set the parameter "withMean" to center the data (i.e., have mean zero). This parameter is by default set to false. If you have a sparse vector and you want to center the data then spark must first convert this sparse vector to a dense vector to subtract the mean from each element.

The following code segment takes a column of feature vector and standardize each feature in the vector.. The method Vectors.dense in the following code segment creates a dense vector.

In [36]:
import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._
val df=Seq((0,Vectors.dense(5.4,125,-1)),(1,Vectors.dense(5.8,200,0)),(0,Vectors.dense(6.0,178,2))).toDF("id","features")
val standardized=new StandardScaler().setWithMean(true).setInputCol("features").setOutputCol("scaled_and_centered").fit(df).transform(df)
println("standardized")
standardized.show(truncate=false)

standardized
+---+----------------+-------------------------------------------------------------+
|id |features        |scaled_and_centered                                          |
+---+----------------+-------------------------------------------------------------+
|0  |[5.4,125.0,-1.0]|[-1.0910894511799611,-1.1067017740229528,-0.8728715609439694]|
|1  |[5.8,200.0,0.0] |[0.21821789023599167,0.8386724381267693,-0.21821789023599236]|
|0  |[6.0,178.0,2.0] |[0.8728715609439696,0.26802933589618416,1.0910894511799618]  |
+---+----------------+-------------------------------------------------------------+



import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._
df: org.apache.spark.sql.DataFrame = [id: int, features: vector]
standardized: org.apache.spark.sql.DataFrame = [id: int, features: vector ... 1 more field]


## MinMaxScaler
MinMaxScaler is an estimator which takes a column of feature vectors and computes its min-max scaling by subtracting the min of each feature and dividing it by its range (max-min). for example 5.8 is scaled as (5.8-5.4)/(6.0 - 5.4)=0.666  

In [35]:
val minmax=new MinMaxScaler().setInputCol("features").setOutputCol("min_max_scaled").fit(df).transform(df)
println("MinMax Scalar")
minmax.show(truncate=false)

MinMax Scalar
+---+----------------+-------------------------------------------+
|id |features        |min_max_scaled                             |
+---+----------------+-------------------------------------------+
|0  |[5.4,125.0,-1.0]|[0.0,0.0,0.0]                              |
|1  |[5.8,200.0,0.0] |[0.6666666666666662,1.0,0.3333333333333333]|
|0  |[6.0,178.0,2.0] |[1.0,0.7066666666666667,1.0]               |
+---+----------------+-------------------------------------------+



minmax: org.apache.spark.sql.DataFrame = [id: int, features: vector ... 1 more field]


## Normalizer
Normalizer is a transformer which takes a column of feature vectors and normalize each vector to have unit norm ( that is dividing each value by its p-norm). It takes parameter p which specifies the p-norm used for normalization (by default p=2 computing the L2 norm). let's take the dataframe "df" we created in the previous section and normalize it.

If you look at the normalized column, you see that each vector is normalized row-wise as opposed to the StandardScaler and MinmaxScaler which are applied column-wise. Hence, spark Nomarlizer is a transformer which is applied to each individual observation while StandardScaler and MinMaxScaler are estimators applied to all observations for each feature.


In [38]:
val normalizer = new Normalizer().setInputCol("features").setOutputCol("normalized").setP(1.0)

val l1NormData = normalizer.transform(df)
println("Normalized using L^1 norm")
l1NormData.show(truncate=false)

Normalized using L^1 norm
+---+----------------+------------------------------------------------------------+
|id |features        |normalized                                                  |
+---+----------------+------------------------------------------------------------+
|0  |[5.4,125.0,-1.0]|[0.0410958904109589,0.9512937595129376,-0.0076103500761035] |
|1  |[5.8,200.0,0.0] |[0.028182701652089404,0.9718172983479105,0.0]               |
|0  |[6.0,178.0,2.0] |[0.03225806451612903,0.956989247311828,0.010752688172043012]|
+---+----------------+------------------------------------------------------------+



normalizer: org.apache.spark.ml.feature.Normalizer = normalizer_eb356b31f084
l1NormData: org.apache.spark.sql.DataFrame = [id: int, features: vector ... 1 more field]


## Bucketizer
is a transformer which takes a continuous feature and bins it according to an array of user-defined splits. The         splits must be provided in an strictly increasing order. If you don't know what are the max and min values in your dataset you could set the lower split to Double.NegativeInfinity and upper split to Double.PositiveInfinity.  
The code segment below take a column "Age" and splits it into five groups: (-INF,1),[1,3),[3,6),[6,13),[13,+INF)



In [43]:
val splits = Array(Double.NegativeInfinity, 1, 3, 6,13, Double.PositiveInfinity)

val AgeData= Seq((2),(5),(7),(12),(15),(2),(0)).toDF("Age")

val bucketizer = new Bucketizer()
  .setInputCol("Age")
  .setOutputCol("AgeGroup")
  .setSplits(splits)

// Transform original data into its bucket index.
val bucketedData = bucketizer.transform(AgeData)
bucketedData.show

+---+--------+
|Age|AgeGroup|
+---+--------+
|  2|     1.0|
|  5|     2.0|
|  7|     3.0|
| 12|     3.0|
| 15|     4.0|
|  2|     1.0|
|  0|     0.0|
+---+--------+



splits: Array[Double] = Array(-Infinity, 1.0, 3.0, 6.0, 13.0, Infinity)
AgeData: org.apache.spark.sql.DataFrame = [Age: int]
bucketizer: org.apache.spark.ml.feature.Bucketizer = bucketizer_5da70f2f29cb
bucketedData: org.apache.spark.sql.DataFrame = [Age: int, AgeGroup: double]


## QuantileDiscretizer
Is an estimator which takes a continuous feature and bins it according to "numBins" parameter provided by user.Each bin have more or less the same number of data points (some bins may have less) and the data points in each bin are bigger than the ones in all its previous bins and smaller than the ones in all its subsequent bins. It is possible that the number of buckets used will be smaller than "numBins" value, for example, if there are too few distinct values of the input to create enough distinct quantiles.

The code segment below takes a numeric column "hour" and divides it into three buckets where each bucket approximately holds 33% of the data points and the smallest hour is mapped to the first bin (bin number 0), the next two hours are mapped to the second bin, and the largest two hours are mapped to the the third bin.

In [45]:
val data = Array((0, 18.0), (1, 19.0), (2, 8.0), (3, 5.0), (4, 2.2))
val hour_data = spark.createDataFrame(data).toDF("id", "hour")

val discretizer = new QuantileDiscretizer().setInputCol("hour").setOutputCol("result").setNumBuckets(3)

val result = discretizer.fit(hour_data).transform(hour_data)
result.show

+---+----+------+
| id|hour|result|
+---+----+------+
|  0|18.0|   2.0|
|  1|19.0|   2.0|
|  2| 8.0|   1.0|
|  3| 5.0|   1.0|
|  4| 2.2|   0.0|
+---+----+------+



data: Array[(Int, Double)] = Array((0,18.0), (1,19.0), (2,8.0), (3,5.0), (4,2.2))
hour_data: org.apache.spark.sql.DataFrame = [id: int, hour: double]
discretizer: org.apache.spark.ml.feature.QuantileDiscretizer = quantileDiscretizer_dda3b6acbf9f
result: org.apache.spark.sql.DataFrame = [id: int, hour: double ... 1 more field]
